In [0]:
# Lab 10 MNIST and Xavier
import tensorflow as tf
import random
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# input place holders
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32, name='dropout_ratio')

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
with tf.name_scope("Layer1"):
  W1 = tf.get_variable("W1", shape=[784, 256],
                       initializer=tf.contrib.layers.xavier_initializer())
  b1 = tf.Variable(tf.random_normal([256]))
  L1 = tf.nn.relu(tf.matmul(x, W1) + b1)
  L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
  
  tf.summary.histogram("W1", W1)
  tf.summary.histogram("b1", b1)
  tf.summary.histogram("L1", L1)
  
with tf.name_scope("Layer2"):
  W2 = tf.get_variable("W2", shape=[256, 256],
                       initializer=tf.contrib.layers.xavier_initializer())
  b2 = tf.Variable(tf.random_normal([256]))
  L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
  L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
  
  tf.summary.histogram("W2", W2)
  tf.summary.histogram("b2", b2)
  tf.summary.histogram("L2", L2)

with tf.name_scope("Layer3"):
  W3 = tf.get_variable("W3", shape=[256, 10],
                       initializer=tf.contrib.layers.xavier_initializer())
  b3 = tf.Variable(tf.random_normal([10]))
  hypothesis = tf.matmul(L2, W3) + b3
  hypothesis = tf.nn.dropout(hypothesis, keep_prob=keep_prob)
  
  tf.summary.histogram("W3", W3)
  tf.summary.histogram("b3", b3)
  tf.summary.histogram("Hypothesis", hypothesis)

# define cost/loss & optimizer
with tf.name_scope("Cost"):
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
      logits=hypothesis, labels=y))  
  tf.summary.scalar("Cost", cost)
  
with tf.name_scope("Train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
# initialize
with tf.Session() as sess:
  merged_summary = tf.summary.merge_all()
  writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
  writer.add_graph(sess.graph)  # Show the graph

  sess.run(tf.global_variables_initializer())

  # train my model
  for epoch in range(training_epochs):
      avg_cost = 0
      total_batch = int(mnist.train.num_examples / batch_size)

      for i in range(total_batch):
          batch_xs, batch_ys = mnist.train.next_batch(batch_size)
          feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 0.7}
          c, summary, _ = sess.run([cost, merged_summary, optimizer], feed_dict=feed_dict)

          writer.add_summary(summary, global_step=i)
          avg_cost += c

      avg_cost /= total_batch
      print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

  print('Learning Finished!')

  # Test model and check accuracy
  correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print('Accuracy:', sess.run(accuracy, feed_dict={
        x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

  # Get one and predict
  r = random.randint(0, mnist.test.num_examples - 1)
  print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
  print("Prediction: ", sess.run(
      tf.argmax(hypothesis, 1), feed_dict={x: mnist.test.images[r:r + 1], keep_prob:1.0}))

#   plt.imshow(mnist.test.images[r:r + 1].
#             reshape(28, 28), cmap='Greys', interpolation='nearest')
#   plt.show()

Epoch: 0001 cost = 0.886344366
Epoch: 0002 cost = 0.585799684
Epoch: 0003 cost = 0.534216212
Epoch: 0004 cost = 0.503948743
Epoch: 0005 cost = 0.482543672
Epoch: 0006 cost = 0.478262219
Epoch: 0007 cost = 0.462819871
Epoch: 0008 cost = 0.466006463
Epoch: 0009 cost = 0.451392857
Epoch: 0010 cost = 0.442938593
Epoch: 0011 cost = 0.446961411
Epoch: 0012 cost = 0.442196406
Epoch: 0013 cost = 0.435610007
Epoch: 0014 cost = 0.430275250
Epoch: 0015 cost = 0.427070228
Learning Finished!
Accuracy: 0.98
Label:  [9]
Prediction:  [9]


***Optimizer***

tf.train.AdadeltaOptimizer
tf.train.AdagradOptimizer
tf.train.AdagradDAOptimizer
tf.train.MomentumOptimizer
tf.train.AdamOptimizer
tf.train.FtrlOptimizer
tf.train.ProximalGradientDescentOptimizer
tf.train.ProximalAdagradOptimizer
tf.train.RMSPropOptimizer
tf.train.GradientDescentOptimizer